1. load data
2. import VGG16 discard FC layer,expand dims, preprocess the input
3. extract characteristics
4. Flatten
5. visualize the data
6. Split the training and test data
7. Construct MLP Model (1 hidden layers 10 units, 1 output layer)
8. 

In [10]:
import os
import random
import shutil
import csv
import numpy as np
def CopyFile(imageDir,test_rate,save_test_dir,save_train_dir):
    image_number = len(imageDir)
    test_number = int(image_number * test_rate)
    print("the number of pic move to %s: %d" %(save_test_dir,test_number))
    test_samples = random.sample(imageDir, test_number)
    if not os.path.exists(save_test_dir):
        os.makedirs(save_test_dir)
        print("save_test_dir has been created successfully!")
    else:
        print("save_test_dir already existed!")
    if not os.path.exists(save_train_dir):
        os.makedirs(save_train_dir)
        print("save_train_dir has been created successfully!")
    else:
        print("save_train_dir already existed!")
    for i,j in enumerate(test_samples):
        shutil.copy(test_samples[i],save_test_dir+test_samples[i].split("/")[-1])
    print("test move success!")
    for train_imgs in imageDir:
        if train_imgs not in test_samples:
            shutil.copy(train_imgs, save_train_dir+train_imgs.split('/')[-1])
    print("train move success!")


file_path = "C:/非系统/dogs-vs-cats/train/train"
test_rate = 0.75

file_dirs = os.listdir(file_path)
origin_dirs = []
save_test_dirs = []
save_train_dirs = []
for path in file_dirs:
    origin_dirs.append(file_path + "/" + path +"/")
    save_test_dirs.append("./test/"+path+"/")
    save_train_dirs.append("./train/"+path+"/")
for i,origin_path in enumerate(origin_dirs):
    image_list = os.listdir(origin_path)
    image_dir = []
    for x,y in enumerate(image_list):
        image_dir.append(os.path.join(origin_path,y))
    print("There are %d pics in dir %s" %(len(image_dir),origin_path))
    CopyFile(image_dir,test_rate,save_test_dirs[i],save_train_dirs[i])
print("All data has been moved successfully!")

There are 12500 pics in dir C:/非系统/dogs-vs-cats/train/train/cat/
the number of pic move to ./test/cat/: 9375
save_test_dir already existed!
save_train_dir already existed!
test move success!
train move success!
There are 12500 pics in dir C:/非系统/dogs-vs-cats/train/train/dog/
the number of pic move to ./test/dog/: 9375
save_test_dir already existed!
save_train_dir already existed!
test move success!
train move success!
All data has been moved successfully!


In [74]:
#expand dims and preprocess the input

# from tensorflow.keras.preprocessing.image import img_to_array, load_img is used for process single img
# or define a picture generator yourself[configure the ways of preprocessing yourself]

from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(rescale =1./255, rotation_range =40, width_shift_range = 0.2, height_shift_range = 0.2, zoom_range = 0.2)
train_data = data_gen.flow_from_directory("./train",target_size = (224,224),batch_size = 32, class_mode = 'binary')
test_data = data_gen.flow_from_directory("./test",target_size = (224,224),batch_size = 32, class_mode = 'binary')

Found 17120 images belonging to 2 classes.
Found 24521 images belonging to 2 classes.


In [76]:
# # for single pic
# from tensorflow.keras.preprocessing.image import img_to_array, load_img
# from tensorflow.keras.applications.vgg16 import preprocess_input
# import numpy as np
# # Since we need to reconfigure the picture

# def preprocessing(img_path):
#     pic_target = img_path
#     pic_target = load_img(pic_target,target_size =(224,224))
#     pic_target = img_to_array(pic_target)
#     return pic_target

# img = preprocessing('./train/cat/cat.4.jpg')
# ##preprocess the input
# x = np.expand_dims(img,axis = 0)   # add the dim before the first dimension
# # otherwise the result x shape will be 224,224,3
# print(type(x))
# print(x.shape)
# x = preprocess_input(x)
# print(type(x))
# print(x.shape)

In [75]:
# # visualize the data

# %matplotlib inline
# import matplotlib.pyplot as plt
# fig = plt.figure(figsize = (5,5))
# img = load_img('./train/cat/cat.4.jpg', target_size = (224,224))
# plt.imshow(img)

In [79]:
# import VGG16 discard FC layer
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.Models import Sequence

model_vgg = VGG16(weights = "imagenet",include_top = False)
x = model_vgg.output
print(x)
#如果要开发新模型，则使用none的权重参数
#include_top 表示是否使用全连接层

Tensor("block5_pool_8/Identity:0", shape=(None, None, None, 512), dtype=float32)


In [103]:
#  批量预处理的方法
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
modelvgg = VGG16(weights = 'imagenet',include_top = False)
def modelPreprocess(img_path,model):
    img = load_img(img_path,target_size=(224,224))
    img = img_to_array(img)
    img = np.expand_dims(img,axis = 0)
    img = preprocess_input(img)
    x = modelvgg.predict(img)
    x = x.reshape(1,7*7*512)
    return x


In [100]:
import os
a = os.listdir("./test/dog/")
for i in a:
    print(type(i))
    break

<class 'str'>


In [ ]:
import os
dirs= ["./test","./train"]
sub_dirs = ["cat","dog"]

feastures = []
for i,dir in enumerate(dirs):
    sub_features = []
    for j,sub_dir in enumerate(sub_dirs):
        curr_path = os.path.join(dir,sub_dir)
        pics = os.listdir(curr_path)
        feature = np.zeros([len(pics),7*7*512])
        for x in range(len(pics)):
            feature = modelPreprocess(curr_path+"/"+pics[x],modelvgg)
            print("Preprocessed: ", pics[x])
            sub_features.append(feature)
    features.append(sub_features)

print(features[0].shape,features[1].shape)
y1 = np.ones(12500)
y1 = np.zeros(12500)
x = np.concatenate((features[0][0],features[1][0],features[0][1],features[1][1]),axis = 0)
y = np.concatenate(y1,y2)
y = y.reshape(-1,1)

In [ ]:
# split the train and test dataset
from sklearn.model.selection import train_test_split 
(x_train, x_test), (y_train,y_test) = train_test_split(x,y,test_size = 0.3,random_state = 50)
print(x_train.shape,x_test.shape,x.shape)

In [108]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Dense(units = 10, activation = 'relu', input_dim = 7*7*512))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                250890    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 250,901
Trainable params: 250,901
Non-trainable params: 0
_________________________________________________________________


In [113]:
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics = 'accuracy')
model.fit(x_train,y_train,epochs = 10)

NameError: name 'x_train' is not defined